In [ ]:
!pip install wget

In [ ]:
import os
import re
from pathlib import Path
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.exceptions import ConvergenceWarning
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import random
import wget
random.seed()
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Please add necessary imports here

In [ ]:
filename = wget.download("https://github.com/MIE1513HS-2022/course-datasets/raw/main/20_newsgroups.zip")
!unzip 20_newsgroups.zip
DATA_DIR = "20_newsgroups"

## Functions from lab

**PLEASE DO NOT CHANGE FUNCTION/CLASS NAMES**

**PLEASE DO NOT CHANGE/OVERWRITE COMPLETED FUNCTIONS**

In [ ]:
def clean_file_text(text):
    new_text = re.sub("Newsgroups:.*?\n", "", text)
    new_text = re.sub("Xref:.*?\n", "", new_text)
    new_text = re.sub("Path:.*?\n", "", new_text)
    new_text = re.sub("Date:.*?\n", "", new_text)
    new_text = re.sub("Followup-To:.*?\n", "", new_text)
    return new_text

In [ ]:
def corpus_count_words(file_list):
    tokenizer = RegexpTokenizer(r'\w+')
    word_counter = Counter()
    for file_path in file_list:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            file_data = file.read()
            file_data = clean_file_text(file_data)
            file_words = tokenizer.tokenize(file_data)
            word_counter.update(file_words)
    return word_counter

In [ ]:
def get_topic_name(file_path):
    return file_path.parent.name

def get_target(topic_name):
    topics = ['talk.politics.mideast', 'rec.autos', 'comp.sys.mac.hardware', 'alt.atheism', 'rec.sport.baseball', 
     'comp.os.ms-windows.misc', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'talk.politics.misc', 
     'rec.motorcycles', 'comp.windows.x', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'sci.electronics',
     'talk.politics.guns', 'sci.space', 'soc.religion.christian', 'misc.forsale', 'talk.religion.misc']
    return topics.index(topic_name)

In [ ]:
def plot_confusion_matrix(cm):
    # plot the confusion matrix
    plt.figure(figsize=(10,10))
    plt.matshow(cm, fignum=1)
    
    # add labels for all targets
    num_targets = cm.shape[0]
    plt.xticks(list(range(num_targets+1)))
    plt.yticks(list(range(num_targets+1)))

## Q1

In [ ]:
class MLQ1():
    def binary_baseline_data(self, file_list, num_words=1000):
        # Calculate word count in corpus
        news_cnt = corpus_count_words(file_list)

        # Select the most common numWords
        word_list = [word for (word, freq) in news_cnt.most_common(num_words)]

        # Create a binary encoding of dataset based on the selected features (X)
        tokenizer = RegexpTokenizer(r'\w+')
        df_rows = []
        for file_path in file_list:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                file_data = file.read()
                file_data = clean_file_text(file_data)
                file_words = tokenizer.tokenize(file_data)
                df_rows.append([1 if word in file_words else 0 for word in word_list])
        X = pd.DataFrame(df_rows, index=[str(f) for f in file_list], columns = word_list)

        # Create a dataframe of targets (y)
        y = [get_target(get_topic_name(file_path)) for file_path in file_list]

        return X, y

    def binary_improved_data(self, file_list, num_words=1000):
        # Put your code here for Q1b
        # Make sure you update the variable features and targets below

        X = None
        y = None

        # Please remember to put index for your dataframe as the file name
        # For example: pd.DataFrame(data, index=[str(f) for f in file_list],columns=[])

        # validate return types
        assert isinstance(X, pd.DataFrame) and isinstance(y, list), "return types"

        return X, y

    def train_and_predict_baseline(self, file_list):
        # get the baseline data
        X, y = self.binary_baseline_data(file_list)

        # split to train and test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # train a logistic regression classifier
        clf = LogisticRegression(C=1.0).fit(X_train, y_train)

        # predict on train and test set
        y_train_predict = clf.predict(X_train)
        y_test_predict = clf.predict(X_test)

        # calculate train and test accuracy
        train_accuracy = accuracy_score(y_train, y_train_predict)
        test_accuracy = accuracy_score(y_test, y_test_predict)

        return X, y, train_accuracy, test_accuracy

    def train_and_predict_improved(self, file_list):
        # get the baseline data
        X, y = self.binary_improved_data(file_list)

        # Write your code here for Q1c:
        # You need to split the data and train a logistic regression classifier.
        # Then, you need to calculate the variables train_accuracy and test_accuracy for the new classifier

        return X, y, train_accuracy, test_accuracy

    @staticmethod
    def random_mean_ci(X, y, num_tests):
        # train_results is a list of train accuracy results for the differrent random splits of the dataset
        train_results = []

        # test_results is a list of test accuracy results for the differrent random splits of the dataset
        test_results = []

        # Write your code here for Q1d

        # calculate the train mean and the 95% confidence interval for the list of results
        train_mean = np.mean(train_results)
        train_ci_low, train_ci_high = stats.t.interval(0.95, len(train_results)-1, loc=train_mean, scale=stats.sem(train_results))

        # calculate the test mean and the 95% confidence interval for the list of results
        test_mean = np.mean(test_results)
        test_ci_low, test_ci_high = stats.t.interval(0.95, len(test_results)-1, loc=test_mean, scale=stats.sem(test_results))

        # validate return types
        assert isinstance(train_mean, float) and isinstance(train_ci_low, float) and isinstance(train_ci_high, float), "return types"
        assert isinstance(test_mean, float) and isinstance(test_ci_low, float) and isinstance(test_ci_high, float), "return types"

        return train_mean, train_ci_low, train_ci_high, test_mean, test_ci_low, test_ci_high

    @staticmethod
    def random_cm(X, y, num_tests):
        # cm_list is a list of confusion matrices for the different random splits of the dataset
        cm_list = []

        # Write your code here for Q1f

        # sum the confusion matrices and return the combined confusion matrix
        combined_cm = np.array(cm_list).sum(axis=0)

        # validate return type
        assert isinstance(combined_cm, np.ndarray), "return type"

        return combined_cm



Please do not change this global variable

In [ ]:
all_files = [pth for pth in Path(DATA_DIR).glob("**/*") if pth.is_file() and not pth.name.startswith(".")]

In [ ]:
q1 = MLQ1()
X, y, train_accuracy, test_accuracy = q1.train_and_predict_baseline(all_files)
# report results
print("Train accuracy: {}".format(train_accuracy))
print("Test accuracy: {}".format(test_accuracy))

### Q1 (a)

[Write your answer here]

### Q1 (b)

Implement *binary_improved_data*

### Q1 (c)

Implement *train_and_predict_improved*

In [ ]:
X, y, train_accuracy, test_accuracy = q1.train_and_predict_improved(all_files)
# report results
print("Train accuracy: {}".format(train_accuracy))
print("Test accuracy: {}".format(test_accuracy))


[Write your answer here]

### Q1 (d)

Implement *random_mean_ci*


### Q1 (e)

Use the following code to calculate the mean accuracy and 95% confidence interval over 10 random splits

In [ ]:
train_mean10, train_low10, train_high10, test_mean10, test_low10, test_high10 = MLQ1.random_mean_ci(X, y, num_tests = 10)
print("Train mean accuracy over 10 random splits: {}".format(train_mean10))
print("Train confidence interval over 10 random splits: [{}, {}]".format(train_low10, train_high10))
print("Test mean accuracy over 10 random splits: {}".format(test_mean10))
print("Test confidence interval over 10 random splits: [{}, {}]".format(test_low10, test_high10))

[ Write your answer here ]

### Q1 (f)

Implement *random_cm*


### Q1 (g)

Use the following code to produce a confusion matrix for 10 random splits

In [ ]:
cm10 = MLQ1.random_cm(X, y, num_tests = 10)
plot_confusion_matrix(cm10)

[ Write your answer here ]

## Q2

In [ ]:
class MLQ2():
    def feature_num(self, X, y):
        # result_list is a list of tuples (num_features, train_accuracy, test_accuracy)
        # where numFeatures is the number of words used as features
        result_list = []

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        for p in [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]:
            subset_size = int(p*X.shape[1])
            X_train_subset = X_train.iloc[:, 0:subset_size]
            X_test_subset = X_test.iloc[:, 0:subset_size]
            # Write your code here to calculate train_accuracy and test_accuracy for the current subset of features
            # ......
            train_accuracy = None
            test_accuracy = None
            # add to result_list
            result_list.append((p, train_accuracy, test_accuracy))

        # Make a dataframe of the results
        result_df = pd.DataFrame(result_list, columns=["num_features", "train_accuracy", "test_accuracy"])

        # validate return type
        assert isinstance(result_df, pd.DataFrame), "return type"

        return result_df

### Q2 (a)

Implement *feature_num*

### Q2 (b)

Use the following code to plot the train and test accuracy for the different feature sets sizes

In [ ]:
q2 = MLQ2()
feature_num_df = q2.feature_num(X, y)
feature_num_df.plot(x="num_features", y=["train_accuracy", "test_accuracy"])

[ Write your answer here ]

## Q3

In [ ]:
class MLQ3():
    def hyperparameter(self, X, y):
        # result_list is a list of tuples (num_features, train_accuracy, test_accuracy)
        # where numFeatures is the number of words used as features
        result_list = []

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        for param in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
            # Write your code here to calculate train_accuracy and test_accuracy for the current parameter value
            # ......
            train_accuracy = None
            test_accuracy = None

            # add to result_list
            result_list.append((param, train_accuracy, test_accuracy))

        # Make a dataframe of the results
        result_df = pd.DataFrame(result_list, columns=["param", "train_accuracy", "test_accuracy"])

        # validate return type
        assert isinstance(result_df, pd.DataFrame), "return type"

        return result_df

### Q3 (a)

Implement *hyperparameter*

### Q3 (b)

Use the following code to plot the train and test accuracy for the different the parameters

In [ ]:
q3 = MLQ3()
param_df = q3.hyperparameter(X, y)
param_df.plot(x="param", y=["train_accuracy", "test_accuracy"], logx=True)

[ Write your answer here ]

## Q4

In [ ]:
class MLQ4():
    def tf_improved_data(self, file_list, num_words = 1000):
        # Put your code here
        # Make sure you update the variable features and targets below

        X = None
        y = None

        # validate return types
        assert isinstance(X, pd.DataFrame) and isinstance(y, list), "return types"

        return X, y


### Q4 (a)

Implement *tf_improved_data*

### Q4 (b)

Use the following code to calculate the mean accuracy and 95% confidence interval over multiple random splits

In [ ]:
q4 = MLQ4()
X_tf, y_tf = q4.tf_improved_data(all_files)
train_mean10, train_low10, train_high10, test_mean10, test_low10, test_high10 = MLQ1.random_mean_ci(X_tf, y_tf, num_tests = 10)
print("Train mean accuracy over 10 random splits: {}".format(train_mean10))
print("Train confidence interval over 10 random splits: [{}, {}]".format(train_low10, train_high10))
print("Test mean accuracy over 10 random splits: {}".format(test_mean10))
print("Test confidence interval over 10 random splits: [{}, {}]".format(test_low10, test_high10))

[ Write your answer here ]

## Q5

In [ ]:
class MLQ5():
    def nb_random_mean_ci(self, X, y, num_tests):
        # train_results is a list of train accuracy results for the differrent random splits of the dataset
        train_results = []

        # test_results is a list of test accuracy results for the differrent random splits of the dataset
        test_results = []

        # Write your code here

        # calculate the train mean and the 95% confidence interval for the list of results
        train_mean = np.mean(train_results)
        train_ci_low, train_ci_high = stats.t.interval(0.95, len(train_results)-1, loc=train_mean, scale=stats.sem(train_results))

        # calculate the test mean and the 95% confidence interval for the list of results
        test_mean = np.mean(test_results)
        test_ci_low, test_ci_high = stats.t.interval(0.95, len(test_results)-1, loc=test_mean, scale=stats.sem(test_results))

        # validate return types
        assert isinstance(train_mean, float) and isinstance(train_ci_low, float) and isinstance(train_ci_high, float), "return types"
        assert isinstance(test_mean, float) and isinstance(test_ci_low, float) and isinstance(test_ci_high, float), "return types"

        return train_mean, train_ci_low, train_ci_high, test_mean, test_ci_low, test_ci_high


### Q5 (a)

Implement *nb_random_mean_ci*


### Q5 (b)

Use the following code to calculate the mean accuracy and 95% confidence interval over multiple random splits

In [ ]:
q5 = MLQ5()
train_mean10, train_low10, train_high10, test_mean10, test_low10, test_high10 = q5.nb_random_mean_ci(X, y, num_tests = 10)
print("Train mean accuracy over 10 random splits: {}".format(train_mean10))
print("Train confidence interval over 10 random splits: [{}, {}]".format(train_low10, train_high10))
print("Test mean accuracy over 10 random splits: {}".format(test_mean10))
print("Test confidence interval over 10 random splits: [{}, {}]".format(test_low10, test_high10))

[ Write your answer here ]

## Q6

In [ ]:
class MLQ6():
    def binary_med_data(self, file_list, num_words = 1000):
        # Put your code here
        # Make sure you update the variable features and targets below

        X = None
        y = None

        #Please remember to put index for your dataframe as the file name
        #For example: pd.DataFrame(data, index=[str(f) for f in file_list],columns=[])

        # validate return types
        assert isinstance(X, pd.DataFrame) and isinstance(y, list), "return types"

        return X, y

### Q6 (a)

Implement *binary_med_data*


### Q6 (b)

Use the following code to calculate the mean accuracy and 95% confidence interval over multiple random splits

In [ ]:
q6 = MLQ6()
X, y = q6.binary_med_data(all_files)
train_mean10, train_low10, train_high10, test_mean10, test_low10, test_high10 = MLQ1.random_mean_ci(X, y, num_tests = 10)
print("Train mean accuracy over 10 random splits: {}".format(train_mean10))
print("Train confidence interval over 10 random splits: [{}, {}]".format(train_low10, train_high10))
print("Test mean accuracy over 10 random splits: {}".format(test_mean10))
print("Test confidence interval over 10 random splits: [{}, {}]".format(test_low10, test_high10))

[ Write your answer here ]